# <div style="color:white;display:fill;border-radius:5px;background-color:#9DCDD1;overflow:hidden"><p style="padding:20px;color:white;overflow:hidden;font-size:100%;letter-spacing:0.5px;margin:0"><b>1 | About</b></div>

Sale forecasting and data enrichment model.

## <div style="color:white;display:fill;border-radius:5px;background-color:#9DCDD1;overflow:hidden"><p style="padding:20px;color:white;overflow:hidden;font-size:100%;letter-spacing:0.5px;margin:0"><b>2 | Data overview</b></div>
- Tabular data (.csv)
- 5 years worth of product sales data
- 4 features
  - ["date", "store_id", "item_id", "sales"]
- limited information (i.e., only date and sales are useful) for our model to understand how to succesfully predict future sales
- forecast the future sales of products for the next 3 months

## <div style="color:white;display:fill;border-radius:5px;background-color:#9DCDD1;overflow:hidden"><p style="padding:20px;color:white;overflow:hidden;font-size:100%;letter-spacing:0.5px;margin:0"><b>3 | Stack</b></div>

- Catboost: state-of-the-art gradient boosting on decision trees 
- Upgini: data enrichment given limited data 
  - used to automatically search through thousands of public data sources to find the most relevant features that work for the project


## <div style="color:white;display:fill;border-radius:5px;background-color:#9DCDD1;overflow:hidden"><p style="padding:20px;color:white;overflow:hidden;font-size:100%;letter-spacing:0.5px;margin:0"><b>4 | Exploratory Data Analysis</b></div>

In [ ]:
!pip install -q upgini catboost

In [12]:
from os.path import exists
import pandas as pd

df_path = "train.csv.zip" if exists("train.csv.zip") else "https://github.com/upgini/upgini/raw/main/notebooks/train.csv.zip"
df = pd.read_csv(df_path)
df = df.sample(n=19_000, random_state=0)
df["store"] = df["store"].astype(str)
df["item"] = df["item"].astype(str)

# Convert date column to datetime pandas object
df["date"] = pd.to_datetime(df["date"])

df.sort_values("date", inplace=True)
df.reset_index(inplace=True, drop=True)
df.head()

,date,store,item,sales
0,2013-01-01,7,5,5
1,2013-01-01,4,9,19
2,2013-01-01,1,33,37
3,2013-01-01,3,41,14
4,2013-01-01,5,24,26


## <div style="color:white;display:fill;border-radius:5px;background-color:#9DCDD1;overflow:hidden"><p style="padding:20px;color:white;overflow:hidden;font-size:100%;letter-spacing:0.5px;margin:0"><b>5 | Training</b></div>

### <b><span style='color:#58A2A8'>5.1</span> | Creating train and test splits</b>

In [13]:
train = df[df["date"] < "2017-01-01"]
test = df[df["date"] >= "2017-01-01"]

In [14]:
train_features = train.drop(columns=["sales"])
train_target = train["sales"]
test_features = test.drop(columns=["sales"])
test_target = test["sales"]

### <b><span style='color:#58A2A8'>5.2</span> | Data enrichment</b>

In [15]:
from upgini import FeaturesEnricher, SearchKey
from upgini.metadata import CVType

enricher = FeaturesEnricher(
    search_keys = {
      "date": SearchKey.DATE,
    },
    cv = CVType.time_series
)
enricher.fit(train_features,
             train_target,
             eval_set=[(test_features, test_target)]
)

Detected task type: ModelTaskType.REGRESSION


Column name,Status,Description
date,All valid,All values in this column are good to go
target,All valid,All values in this column are good to go


Running search request with search_id=085aa5ce-17a7-4149-9263-3f41afb734b8
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com
Done


33 relevant feature(s) found with the search keys: ['date'].


,feature_name,shap_value,coverage %,type
0,item,0.488097,100.0,CHARACTER
1,store,0.172502,100.0,CHARACTER
2,f_weather_pca_0_94efd18d,0.056469,100.0,NUMERIC
3,f_week_sin1_a71d22f6,0.044848,100.0,NUMERIC
4,f_week_cos1_d3d56d7f,0.029807,100.0,NUMERIC
5,f_weather_umap_48_66a91289,0.025439,100.0,NUMERIC
6,f_weather_umap_24_409427e4,0.018721,100.0,NUMERIC
7,f_weather_umap_33_b9760f68,0.014879,100.0,NUMERIC
8,f_year_cos1_cd165f8c,0.011822,100.0,NUMERIC
9,f_dow_jones_89547e1d,0.007005,100.0,NUMERIC


### <b><span style='color:#58A2A8'>5.3</span> | Model creation</b>

In [16]:
from catboost import CatBoostRegressor
from catboost.utils import eval_metric
model = CatBoostRegressor(verbose=False, allow_writing_files=False, random_state=0)

# Calculate metrics before and after enrichment with new relevant features
enricher.calculate_metrics(
    train_features, train_target, 
    eval_set = [(test_features, test_target)],
    estimator = model,
    scoring = "mean_absolute_percentage_error"
)

Calculating metrics...
Done


,match_rate,baseline mean_absolute_percentage_error,enriched mean_absolute_percentage_error,uplift
,,,,
train,100.0,0.255844,0.170663,0.085181
eval 1,100.0,0.243877,0.131175,0.112701


In [17]:
enriched_train_features = enricher.transform(train_features, keep_input = True)
enriched_test_features = enricher.transform(test_features, keep_input = True)
enriched_train_features.head()

90.39637% of the rows are fully duplicated


Column name,Status,Description
date,All valid,All values in this column are good to go


Running search request with search_id=03ec4887-97a2-4877-a6a6-81d92edffff0
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com
Done

Done
90.36176% of the rows are fully duplicated


Column name,Status,Description
date,All valid,All values in this column are good to go


Running search request with search_id=615b6326-2919-4dc5-baee-0c1114d73e59
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com
Done

Done


,date,store,item,f_weather_pca_0_94efd18d,f_week_sin1_a71d22f6,f_week_cos1_d3d56d7f,f_weather_umap_48_66a91289,f_weather_umap_24_409427e4,f_weather_umap_33_b9760f68,f_year_cos1_cd165f8c,...,f_weather_umap_45_b348f420,f_c2c_fraud_score_5028232e,f_mlending_approve_score_d4c33397,f_silver_7d_to_7d_1y_shift_ccbd2abf,f_cbpol_umap_6_f175da9a,f_cbpol_pca_3_2e94b9bf,f_cpi_umap_4_83d3b2a6,f_transaction_fraud_union_score_c1a2808b,f_bank_approve_score_f3797f4b,f_gold_7d_to_1y_1df66550
0,2013-01-01,7,5,29.676683,0.781831,0.62349,4.540985,5.828106,4.644803,0.98522,...,4.923654,0.285548,0.338412,1.072025,1.367325,-0.323471,10.153208,0.057741,0.332893,0.994608
1,2013-01-01,4,9,29.676683,0.781831,0.62349,4.540985,5.828106,4.644803,0.98522,...,4.923654,0.285548,0.338412,1.072025,1.367325,-0.323471,10.153208,0.057741,0.332893,0.994608
2,2013-01-01,1,33,29.676683,0.781831,0.62349,4.540985,5.828106,4.644803,0.98522,...,4.923654,0.285548,0.338412,1.072025,1.367325,-0.323471,10.153208,0.057741,0.332893,0.994608
3,2013-01-01,3,41,29.676683,0.781831,0.62349,4.540985,5.828106,4.644803,0.98522,...,4.923654,0.285548,0.338412,1.072025,1.367325,-0.323471,10.153208,0.057741,0.332893,0.994608
4,2013-01-01,5,24,29.676683,0.781831,0.62349,4.540985,5.828106,4.644803,0.98522,...,4.923654,0.285548,0.338412,1.072025,1.367325,-0.323471,10.153208,0.057741,0.332893,0.994608


## <div style="color:white;display:fill;border-radius:5px;background-color:#9DCDD1;overflow:hidden"><p style="padding:20px;color:white;overflow:hidden;font-size:100%;letter-spacing:0.5px;margin:0"><b>6 | Performance and Evaluation</b></div>

In [18]:
model.fit(train_features, train_target)
preds = model.predict(test_features)
eval_metric(test_target.values, preds, "SMAPE")

[37.65141857448004]

In [19]:
model.fit(enriched_train_features, train_target)
enriched_preds = model.predict(enriched_test_features)
eval_metric(test_target.values, enriched_preds, "SMAPE")

[14.520546401124]